# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [ ]:
from sklearn.model_selection import train_test_split

X = adult_dt.iloc[:, :-1]
Y = adult_dt['income'] 
Y_df = pd .DataFrame(Y)

type(X), type(Y)  # for debugging purpose

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify= Y,  random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

**Random state is essentilly a randon seed generated by pseudo random generatotr, and it is useful because using the same random state gurantees that the result, here, of splitting, is reproducible.**

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Identify categorical and numerical columns
numerical_cols = X.select_dtypes(include=['int', 'float']).columns
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols,   categorical_cols  # for debugging purpose


# Define preprocessing steps for numerical and categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transform', Pipeline(steps=[
            ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
            ('scaler', RobustScaler())
        ]), numerical_cols),
        
        ('cat_transform', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ]
)
preprocessor

ColumnTransformer(transformers=[('num_transform',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                ('cat_transform',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [8]:
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
    
])
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [9]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(
    pipe, X_train, Y_train, 
    cv = 5,
    scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy'],
    return_train_score=True
)

# Display fold-level results
import pandas as pd

fold_results = pd.DataFrame(cv_results).sort_values(by='test_neg_log_loss')
print(fold_results)

# Calculate the mean of each metric
mean_results = fold_results.mean()
print("Mean results are :\n", mean_results)

    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
3  23.158968    0.549615          -0.394526           -0.081720      0.895921   
1  20.421841    0.466800          -0.387537           -0.082680      0.900172   
4  22.526125    0.297464          -0.366098           -0.081936      0.897342   
2  20.926661    0.461180          -0.356092           -0.083228      0.908136   
0  22.541819    0.306103          -0.350724           -0.081935      0.899779   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
3            1.0       0.849934        0.999945                0.764686   
1            1.0       0.852161        0.999945                0.771427   
4            1.0       0.844669        1.000000                0.758420   
2            1.0       0.856516        1.000000                0.781613   
0            1.0       0.850844        0.999945                0.768073   

   train_balanced_accuracy  
3                 0.999886  
1   

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [10]:
fold_results_df =  pd. DataFrame(fold_results)
fold_results_df

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
3,23.158968,0.549615,-0.394526,-0.081720,0.895921,1.0,0.849934,0.999945,0.764686,0.999886
1,20.421841,0.466800,-0.387537,-0.082680,0.900172,1.0,0.852161,0.999945,0.771427,0.999886
4,22.526125,0.297464,-0.366098,-0.081936,0.897342,1.0,0.844669,1.000000,0.758420,1.000000
2,20.926661,0.461180,-0.356092,-0.083228,0.908136,1.0,0.856516,1.000000,0.781613,1.000000
0,22.541819,0.306103,-0.350724,-0.081935,0.899779,1.0,0.850844,0.999945,0.768073,0.999886


Calculate the mean of each metric. 

In [11]:
Mean_results_df = pd. DataFrame(mean_results)
Mean_results_df

,0
fit_time,21.915083
score_time,0.416232
test_neg_log_loss,-0.370995
train_neg_log_loss,-0.082300
test_roc_auc,0.900270
train_roc_auc,1.000000
test_accuracy,0.850825
train_accuracy,0.999967
test_balanced_accuracy,0.768844
train_balanced_accuracy,0.999932


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [12]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
# Fit the pipeline on the entire training data
pipe.fit(X_train, Y_train)


# Predict probabilities on the test set
Y_pred_proba = pipe.predict_proba(X_test)


# Calculate performance metrics on the test set
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, pipe.predict(X_test)),
    'balanced_accuracy': balanced_accuracy_score(Y_test, pipe.predict(X_test))
}

print("Test metrics:\n", test_metrics)

Test metrics:
 {'neg_log_loss': -0.3734534810826036, 'roc_auc': 0.9060708060357756, 'accuracy': 0.8553587880028662, 'balanced_accuracy': 0.7784459240080015}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

***1 : Converts the boolean array to a numeric array 1 for True, 0 for False. This creates a binary representation of income categories (>50K or not)
This allows us to work directly with numerical outputs and simplify the calculations for performance metrics such as log loss and ROC AUC.**

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.